# CNN - FashionMNIST

### Import Libraries

In [ ]:
import time
import numpy as np
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_data_format("channels_last")

import matplotlib.pyplot as plt
from livelossplot import PlotLossesKeras

### Methods Definition

In [ ]:
# View 6 images of FashionMNIST in a gray scale
def visualize_fashion_mnist(X_train, y_train, X_test, y_test):
    plt.subplot(321)
    plt.imshow(X_train[0], cmap=plt.get_cmap('gray'))
    plt.subplot(322)
    plt.imshow(X_train[1], cmap=plt.get_cmap('gray'))
    plt.subplot(323)
    plt.imshow(X_train[2], cmap=plt.get_cmap('gray'))
    plt.subplot(324)
    plt.imshow(X_train[3], cmap=plt.get_cmap('gray'))
    plt.subplot(325)
    plt.imshow(X_train[4], cmap=plt.get_cmap('gray'))
    plt.subplot(326)
    plt.imshow(X_train[5], cmap=plt.get_cmap('gray'))
    plt.show()


# Useful to visualize the tipology of the network in pdf or png
def print_model(model, fich):
    from keras.utils import plot_model
    plot_model(model, to_file=fich, show_shapes=True, show_layer_names=True)


# Utils to visualize the history of the learning
def print_history_accuracy(history):
    print(history.history.keys())
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()


def print_history_loss(history):
    print(history.history.keys())
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()


def create_compile_model_cnn_simple(num_classes):
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


def create_compile_model_cnn_plus(num_classes):
    model = Sequential()
    model.add(Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(15, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


def fashion_mnist_using_cnn_simple(X_train, y_train, X_test, y_test):

    # Transform to the format [instances][width][height][pixels]
    # (60000, 28, 28)
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
    X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')
    # (60000, 28, 28, 1)

    # Normalize the pixel values from 0-255 to 0-1
    X_train = X_train / 255
    X_test = X_test / 255

    # Transform the label which is an integer into binary categories
    # The value becomes that corresponding to the position, class 5 becomes the list [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)
    num_classes = y_test.shape[1]

    # Define the network topology and compile
    model = create_compile_model_cnn_simple(num_classes)
    #print(model.summary())
    print_model(model, "model_fashion_mnist_cnn_simples.png")
    plotlosses = PlotLossesKeras()

    # Training the network
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2, callbacks=[PlotLossesKeras()]) 
    
    #print_history_accuracy(history)
    #print_history_loss(history)

    # Final evaluation with the test cases
    scores = model.evaluate(X_test, y_test, verbose=0)
    #print('Scores: ', scores)
    print("CNN model accuracy FashionMNIST simple: %.2f%%" % (scores[1]*100))
    print("CNN model error FashionMNIST simple: %.2f%%" % (100-scores[1]*100))


def fashion_mnist_using_cnn_plus(X_train, y_train, X_test, y_test):

    # Transform to the format [instances][width][height][pixels]
    # (60000, 28, 28)
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
    X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')
    # (60000, 28, 28, 1)

    # Normalize the pixel values from 0-255 to 0-1
    X_train = X_train / 255
    X_test = X_test / 255

    # Transform the label which is an integer into binary categories
    # The value becomes that corresponding to the position, class 5 becomes the list [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)
    num_classes = y_test.shape[1]

    # Define the network topology and compile
    model = create_compile_model_cnn_plus(num_classes)
    #print(model.summary())
    print_model(model, "model_fashion_mnist_cnn_plus.png")
    plotlosses = PlotLossesKeras()

    # Training the network
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2, callbacks=[PlotLossesKeras()])
    
    #print_history_accuracy(history)
    #print_history_loss(history)

    # Final evaluation with the test cases
    scores = model.evaluate(X_test, y_test, verbose=0)
    #print('Scores: ', scores)
    print("CNN model accuracy FashionMNIST plus: %.2f%%" % (scores[1]*100))
    print("CNN model error FashionMNIST plus: %.2f%%" % (100-scores[1]*100))

### Load Data

In [ ]:
((x_train, y_train), (x_test, y_test)) = fashion_mnist.load_data()

### Visualize Data

In [ ]:
visualize_fashion_mnist(x_train, y_train, x_test, y_test)

### Model Execution

In [ ]:
start_time = time.time()
fashion_mnist_using_cnn_simple(x_train, y_train, x_test, y_test)
time_taken = time.time() - start_time

print("Processing Time: %.3f seconds (%.3f)" % (time_taken, time_taken/60))

In [ ]:
start_time = time.time()
fashion_mnist_using_cnn_plus(x_train, y_train, x_test, y_test)
time_taken = time.time() - start_time

print("Processing Time: %.3f seconds (%.3f)" % (time_taken, time_taken/60))